# Product Recommendation Model with GPU Training (Workspaces)

This notebook demonstrates GPU-accelerated ML model training in **Snowflake Notebooks in Workspaces**.

**Use Case:** Predict which products a customer is likely to purchase next based on their order history

**What we'll do:**
1. Set execution context for Workspaces
2. Load customer order data from Snowflake
3. Engineer features from purchase patterns
4. Train XGBoost model with GPU acceleration
5. Evaluate model performance
6. Save model to Snowflake Model Registry

**Prerequisites:**
- Run in Snowflake Notebooks in Workspaces with GPU compute pool
- Data in AUTOMATED_INTELLIGENCE.RAW.ORDERS and ORDER_ITEMS

## 1. Set Execution Context

Notebooks in Workspaces don't automatically set database or schema. We need to explicitly set the execution context.

In [3]:
# Standard libraries
import pandas as pd
import numpy as np
import datetime
import time
import warnings
 
# Snowpark for session management and data processing
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import (
    col, count, count_distinct, sum as sum_, avg, max as max_, min as min_,
    datediff, current_date, stddev, coalesce, lit, when
)

# Snowflake ML Registry
from snowflake.ml.registry import Registry

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)
import xgboost as xgb

warnings.filterwarnings('ignore')

# Get active Snowpark session
session = get_active_session()

print("✅ Libraries imported successfully")
print(f"   XGBoost version: {xgb.__version__}")
print(f"   Pandas version: {pd.__version__}")
print(f"\n🚀 Ready for GPU-accelerated training!")

In [4]:
# Set execution context using Snowpark
session.use_database('AUTOMATED_INTELLIGENCE')
session.use_schema('RAW')
session.use_warehouse('COMPUTE_WH')  # Adjust to your warehouse name

# Verify context
context_df = session.sql("""
    SELECT CURRENT_DATABASE() as database, 
           CURRENT_SCHEMA() as schema, 
           CURRENT_WAREHOUSE() as warehouse,
           CURRENT_ROLE() as role
""")

print("✅ Execution context set:")
context_df.show()

## 4. Load and Explore Data with Snowpark

Query orders and order items using Snowpark DataFrames with fully qualified table names.

In [6]:
# Check data volume using Snowpark
print("📊 Checking data volume...\n")

# Query orders table
orders_df = session.table('AUTOMATED_INTELLIGENCE.RAW.ORDERS')
orders_stats = orders_df.select(
    lit('ORDERS').alias('TABLE_NAME'),
    count('*').alias('ROW_COUNT'),
    count_distinct('CUSTOMER_ID').alias('UNIQUE_CUSTOMERS')
)

# Query order items table
order_items_df = session.table('AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS')
items_stats = order_items_df.select(
    lit('ORDER_ITEMS').alias('TABLE_NAME'),
    count('*').alias('ROW_COUNT'),
    count_distinct('ORDER_ID').alias('UNIQUE_ORDERS')
)

# Union and display
data_stats = orders_stats.union(items_stats)
print("Data Volume:")
data_stats.show()

## 5. Feature Engineering

Create customer and product features for recommendation model.

In [7]:
# Step 1: Create customer statistics using Snowpark
print("📊 Step 1: Creating customer statistics...")

# Read orders and order items
orders = session.table('AUTOMATED_INTELLIGENCE.RAW.ORDERS')
order_items = session.table('AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS')

# Join and aggregate using explicit DataFrame references to avoid ambiguity
customer_stats = orders.join(
    order_items, 
    orders['ORDER_ID'] == order_items['ORDER_ID']
).group_by(orders['CUSTOMER_ID']).agg(
    count_distinct(orders['ORDER_ID']).alias('TOTAL_PURCHASES'),
    sum_(order_items['LINE_TOTAL']).alias('TOTAL_SPENT'),
    avg(order_items['LINE_TOTAL']).alias('AVG_ITEM_SPEND'),
    count_distinct(order_items['PRODUCT_ID']).alias('UNIQUE_PRODUCTS'),
    max_(orders['ORDER_DATE']).alias('LAST_PURCHASE_DATE'),
    datediff('day', max_(orders['ORDER_DATE']), current_date()).alias('DAYS_SINCE_LAST_PURCHASE')
)

# Save as temp table
customer_stats.write.mode('overwrite').save_as_table('customer_stats', table_type='temporary')

print(f"✅ Created customer_stats table with {customer_stats.count():,} customers")

In [8]:
# Step 2: Create customer-product pairs using Snowpark
print("📊 Step 2: Creating customer-product purchase history...")

# Join and aggregate customer-product pairs using explicit DataFrame references
customer_products = orders.join(
    order_items,
    orders['ORDER_ID'] == order_items['ORDER_ID']
).group_by([orders['CUSTOMER_ID'], order_items['PRODUCT_ID']]).agg(
    count(lit(1)).alias('PURCHASE_COUNT'),
    sum_(order_items['QUANTITY']).alias('TOTAL_QUANTITY'),
    max_(orders['ORDER_DATE']).alias('LAST_PRODUCT_PURCHASE')
)

# Save as temp table
customer_products.write.mode('overwrite').save_as_table('customer_products', table_type='temporary')

print(f"✅ Created customer_products table with {customer_products.count():,} customer-product pairs")

In [9]:
%%sql
-- Step 3: Create large training dataset to maximize GPU utilization
-- Generates ~5M training samples with rich features
CREATE OR REPLACE TEMPORARY TABLE training_features AS
WITH customer_history AS (
    SELECT 
        o.CUSTOMER_ID,
        COUNT(DISTINCT o.ORDER_ID) as TOTAL_PAST_ORDERS,
        SUM(oi.LINE_TOTAL) as TOTAL_SPENT,
        AVG(oi.LINE_TOTAL) as AVG_ITEM_SPEND,
        COUNT(DISTINCT oi.PRODUCT_ID) as UNIQUE_PRODUCTS_BOUGHT,
        DATEDIFF(day, MAX(o.ORDER_DATE), CURRENT_DATE()) as DAYS_SINCE_LAST_ORDER,
        LISTAGG(DISTINCT oi.PRODUCT_ID, ',') as PURCHASED_PRODUCTS
    FROM AUTOMATED_INTELLIGENCE.RAW.ORDERS o
    JOIN AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS oi ON o.ORDER_ID = oi.ORDER_ID
    GROUP BY o.CUSTOMER_ID
),
product_popularity AS (
    SELECT 
        PRODUCT_ID,
        COUNT(DISTINCT ORDER_ID) as TIMES_ORDERED,
        AVG(UNIT_PRICE) as AVG_PRICE,
        SUM(QUANTITY) as TOTAL_QUANTITY_SOLD,
        STDDEV(UNIT_PRICE) as PRICE_VARIANCE
    FROM AUTOMATED_INTELLIGENCE.RAW.ORDER_ITEMS
    GROUP BY PRODUCT_ID
),
positive_examples AS (
    SELECT 
        cp.CUSTOMER_ID,
        cp.PRODUCT_ID,
        ch.TOTAL_PAST_ORDERS,
        ch.TOTAL_SPENT,
        ch.AVG_ITEM_SPEND,
        ch.UNIQUE_PRODUCTS_BOUGHT,
        ch.DAYS_SINCE_LAST_ORDER,
        pp.TIMES_ORDERED as PRODUCT_POPULARITY,
        pp.AVG_PRICE as PRODUCT_PRICE,
        pp.TOTAL_QUANTITY_SOLD as PRODUCT_VOLUME,
        COALESCE(pp.PRICE_VARIANCE, 0) as PRODUCT_PRICE_VARIANCE,
        1 as PURCHASED
    FROM customer_products cp
    JOIN customer_history ch ON cp.CUSTOMER_ID = ch.CUSTOMER_ID
    JOIN product_popularity pp ON cp.PRODUCT_ID = pp.PRODUCT_ID
),
negative_examples AS (
    SELECT 
        ch.CUSTOMER_ID,
        pp.PRODUCT_ID,
        ch.TOTAL_PAST_ORDERS,
        ch.TOTAL_SPENT,
        ch.AVG_ITEM_SPEND,
        ch.UNIQUE_PRODUCTS_BOUGHT,
        ch.DAYS_SINCE_LAST_ORDER,
        pp.TIMES_ORDERED as PRODUCT_POPULARITY,
        pp.AVG_PRICE as PRODUCT_PRICE,
        pp.TOTAL_QUANTITY_SOLD as PRODUCT_VOLUME,
        COALESCE(pp.PRICE_VARIANCE, 0) as PRODUCT_PRICE_VARIANCE,
        0 as PURCHASED
    FROM customer_history ch
    CROSS JOIN product_popularity pp
    LEFT JOIN customer_products cp 
        ON ch.CUSTOMER_ID = cp.CUSTOMER_ID 
        AND pp.PRODUCT_ID = cp.PRODUCT_ID
    WHERE cp.PRODUCT_ID IS NULL
)
SELECT * FROM positive_examples
UNION ALL
SELECT * FROM negative_examples;

-- Check class balance
SELECT 
    'Total Samples' as metric,
    COUNT(*)::VARCHAR as value
FROM training_features
UNION ALL
SELECT 
    'Class: ' || PURCHASED as metric,
    COUNT(*)::VARCHAR as value
FROM training_features 
GROUP BY PURCHASED
ORDER BY metric;

In [10]:
-- Preview features
SELECT * FROM training_features LIMIT 10;

## 6. Load Data into Pandas for Training

Load training features from Snowflake table.

In [11]:
# Load data from Snowflake (imports already done at top)
print("📊 Loading data from Snowflake...")
start_load = time.time()

# Load data from Snowflake table
pdf = session.sql("SELECT * FROM training_features").to_pandas()

load_time = time.time() - start_load

print(f"\n✅ Data loaded in {load_time:.2f} seconds")
print(f"   Rows: {len(pdf):,}")
print(f"   Columns: {len(pdf.columns)}")

# Preview
print("\n📋 Data Preview:")
pdf.head()

## 7. Encode Categorical Features

In [12]:
# Encode customer_id and product_id
customer_encoder = LabelEncoder()
product_encoder = LabelEncoder()

pdf['CUSTOMER_ID_ENCODED'] = customer_encoder.fit_transform(pdf['CUSTOMER_ID'])
pdf['PRODUCT_ID_ENCODED'] = product_encoder.fit_transform(pdf['PRODUCT_ID'])

# Define features for training (added more features for complexity)
feature_columns = [
    'CUSTOMER_ID_ENCODED',
    'PRODUCT_ID_ENCODED',
    'TOTAL_PAST_ORDERS',
    'TOTAL_SPENT',
    'AVG_ITEM_SPEND',
    'UNIQUE_PRODUCTS_BOUGHT',
    'DAYS_SINCE_LAST_ORDER',
    'PRODUCT_POPULARITY',
    'PRODUCT_PRICE',
    'PRODUCT_VOLUME',
    'PRODUCT_PRICE_VARIANCE'
]

label_column = 'PURCHASED'

# Prepare X and y
X = pdf[feature_columns]
y = pdf[label_column]

print(f"📊 Features shape: {X.shape}")
print(f"   Training samples: {len(X):,}")
print(f"   Features: {len(feature_columns)}")
print(f"   Label distribution: {y.value_counts().to_dict()}")
print(f"\n✅ Features encoded successfully")

## 8. Train XGBoost Model with GPU

**GPU Training with XGBoost:**
- `tree_method='gpu_hist'`: GPU-accelerated histogram-based training
- `predictor='gpu_predictor'`: GPU-accelerated predictions
- Significantly faster training on large datasets

In [13]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Data Split:")
print(f"   Training set: {len(X_train):,} samples")
print(f"   Test set: {len(X_test):,} samples")

In [14]:
# Configure model parameters (imports already done at top)
# High complexity parameters for GPU training
common_params = {
    'n_estimators': 1000,          # 1000 trees for substantial workload
    'max_depth': 20,               # Deep trees for complexity
    'learning_rate': 0.03,         # Lower learning rate for accuracy
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,                  # Regularization
    'reg_alpha': 0.1,              # L1 regularization
    'reg_lambda': 1.0,             # L2 regularization
    'random_state': 42,
    'eval_metric': 'logloss'
}

print("="*70)
print("🚀 GPU-ACCELERATED ML TRAINING")
print("="*70)
print(f"\n📊 Dataset:")
print(f"   Training samples: {len(X_train):,}")
print(f"   Test samples: {len(X_test):,}")
print(f"   Features: {len(feature_columns)}")
print(f"\n📋 Model Configuration:")
print(f"   Trees: {common_params['n_estimators']}")
print(f"   Max Depth: {common_params['max_depth']}")
print(f"   Learning Rate: {common_params['learning_rate']}")
print(f"   Regularization: L1={common_params['reg_alpha']}, L2={common_params['reg_lambda']}")
print("\n" + "="*70)

gpu_params = {**common_params, **{
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
}}

model = xgb.XGBClassifier(**gpu_params)

print(f"⏳ GPU training started at {time.strftime('%H:%M:%S')}...\n")
start_time = time.time()

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=200  # Print every 200 trees
)

training_time = time.time() - start_time

print(f"\n✅ GPU training complete at {time.strftime('%H:%M:%S')}!")
print(f"\n⚡ GPU Performance:")
print(f"   Training time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"   Trees per second: {common_params['n_estimators']/training_time:.2f}")
print(f"   Samples per second: {len(X_train)/training_time:,.0f}")

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🎯 Model Accuracy: {accuracy:.4f}")

## 9. Evaluate Model Performance

In [15]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Classification report
print("📊 Model Performance:")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not Purchased', 'Purchased']))

# Calculate accuracy metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"\n📈 Summary Metrics:")
print(f"   Accuracy: {accuracy:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1 Score: {f1:.4f}")

# Business interpretation
print(f"\n💼 Business Impact:")
print(f"   If we recommend products to customers:")
print(f"   - {precision*100:.1f}% of recommendations will be relevant")
print(f"   - We'll capture {recall*100:.1f}% of products customers want")

## 10. Feature Importance Analysis

In [16]:
# Get feature importance
feature_importance = model.feature_importances_

# Create DataFrame for visualization
feature_importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("🔍 Feature Importance:")
print(feature_importance_df.to_string(index=False))

print("\n💡 Key Insights:")
top_3_features = feature_importance_df.head(3)
for idx, row in top_3_features.iterrows():
    print(f"   {row['feature']}: {row['importance']:.2%}")

## 11. Save Model to Snowflake Model Registry

**Note:** Requires Snowflake ML package and appropriate permissions.

In [19]:
# Initialize Model Registry (imports already done at top)
registry = Registry(
    session=session,
    database_name="AUTOMATED_INTELLIGENCE",
    schema_name="MODELS"
)

# Prepare model metadata
model_name = "product_recommendation_xgboost"
version_name = f"v_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_spcs"

# Sample input for schema inference
sample_input = X_test.head(5)

# Save model
print(f"💾 Saving model to registry...")
print(f"   Model: {model_name}")
print(f"   Version: {version_name}")

try:
    mv = registry.log_model(
        model=model,
        model_name=model_name,
        version_name=version_name,
        sample_input_data=sample_input,
        target_platforms=["SNOWPARK_CONTAINER_SERVICES"],
        conda_dependencies=["xgboost", "scikit-learn", "pandas", "numpy"],     
        comment=f"XGBoost product recommender. Trained on {len(X_train):,} customer-product pairs. F1: {f1:.4f}",
        metrics={
            "accuracy": float(accuracy),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1),
            "training_samples": int(len(X_train)),
            "test_samples": int(len(X_test)),
            "training_time_seconds": float(training_time)
        }
    )

    # Create SPCS service and deploy to system GPU pool
    mv.create_service(
        service_name="gpu_xgboost_service",
        service_compute_pool="SYSTEM_COMPUTE_POOL_GPU",
        ingress_enabled=True,
        gpu_requests="1"  # Enable GPU for gpu_predictor
    )

    print(f"\n✅ Model saved successfully!")
    print(f"\n📦 Model Details:")
    print(f"   Database: AUTOMATED_INTELLIGENCE")
    print(f"   Schema: MODELS")
    print(f"   Model: {model_name}")
    print(f"   Version: {version_name}")
    print(f"   Training Time: {training_time:.2f}s")
except Exception as e:
    print(f"⚠️  Error saving model: {e}")
    print("   Check that AUTOMATED_INTELLIGENCE.MODELS schema exists and you have write permissions")

## 12. Test Model Predictions from Registry

Load the registered model and generate product recommendations for sample customers.

In [20]:
# Load model from registry (reuse registry object from previous cell)
print("📦 Attempting to load model from Snowflake Model Registry...")

try:
    # Get all versions and sort by creation time to get the most recent
    model_obj = registry.get_model('product_recommendation_xgboost')
    versions = model_obj.show_versions()
    
    # Sort by created_on descending to get latest
    versions = versions.sort_values('created_on', ascending=False)
    
    # Get the latest version (first after sorting)
    if len(versions) > 0:
        latest_version = versions.iloc[0]['name']
        model_ref = model_obj.version(latest_version)
        loaded_model = model_ref.load()
        
        print(f"\n✅ Model loaded from registry: product_recommendation_xgboost")
        print(f"   Version: {latest_version} (latest)")
        using_registry = True
    else:
        raise ValueError("No versions found")
except Exception as e:
    print(f"⚠️  Error loading from registry: {e}")
    print("   Using freshly trained model instead")
    loaded_model = model
    using_registry = False

print(f"\n🔮 Generating predictions...\n")

# Get predictions for test set
X_sample = X_test.head(20)
predictions_proba = loaded_model.predict_proba(X_sample)[:, 1]

# Create results DataFrame
results = X_sample.copy()
results['PURCHASE_PROBABILITY'] = predictions_proba
results['PREDICTION'] = (predictions_proba > 0.5).astype(int)
results['ACTUAL'] = y_test.iloc[:20].values

# Sort by probability
results = results.sort_values('PURCHASE_PROBABILITY', ascending=False)

print("🔮 Top Product Recommendations:")
print(results[['CUSTOMER_ID_ENCODED', 'PRODUCT_ID_ENCODED', 
               'PURCHASE_PROBABILITY', 'PREDICTION', 'ACTUAL']].to_string(index=False))

# Calculate recommendation accuracy
correct = (results['PREDICTION'] == results['ACTUAL']).sum()
total = len(results)
print(f"\n✅ Recommendation Accuracy: {correct}/{total} ({correct/total*100:.1f}%)")

if using_registry:
    print(f"\n💡 Predictions made using model from Model Registry!")
else:
    print(f"\n💡 Predictions made using trained model (save to registry to test loading)")